In [1]:
!cp -r ../input/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./
%config Completer.use_jedi = False
%load_ext Cython
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [2]:
URM_all_dataframe = pd.read_csv('/kaggle/input/urm-true-binary/URM_True_Binary.csv')
URM_all_dataframe
URM_all = sps.coo_matrix((URM_all_dataframe["Data"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)
URM_all

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [3]:
URM_train_dataframe = pd.read_csv('/kaggle/input/urm-split/Train_df.csv')
URM_train_dataframe
URM_train = sps.coo_matrix((URM_train_dataframe["Data"].values, 
                          (URM_train_dataframe["UserID"].values, URM_train_dataframe["ItemID"].values)))
URM_train = URM_train.tocsr() # to obtain fast access to rows (users)
URM_train

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1243712 stored elements in Compressed Sparse Row format>

In [4]:
URM_val_dataframe = pd.read_csv('/kaggle/input/urm-split/Test_df.csv')
URM_val_dataframe
URM_valid = sps.coo_matrix((URM_val_dataframe["Data"].values, 
                          (URM_val_dataframe["UserID"].values, URM_val_dataframe["ItemID"].values)))
URM_valid = URM_all.tocsr() # to obtain fast access to rows (users)
URM_valid

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [5]:
from Evaluation.Evaluator import EvaluatorHoldout

#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

In [6]:
import numpy as np
import scipy.sparse as sps
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
import time, sys
from tqdm import tqdm
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# os.environ["PYTHONWARNINGS"] = ('ignore::exceptions.ConvergenceWarning:sklearn.linear_model')
# os.environ["PYTHONWARNINGS"] = ('ignore:Objective did not converge:ConvergenceWarning:')

class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available
    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    @ignore_warnings(category=ConvergenceWarning)
    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100,**earlystopping_kwargs):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)

        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]

            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1

            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)

In [7]:
import implicit
import numpy as np

from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender


class IALSRecommender_implicit(BaseMatrixFactorizationRecommender):
    """
    ALS implemented with implicit following guideline of
    https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe
    IDEA:
    Recomputing x_{u} and y_i can be done with Stochastic Gradient Descent, but this is a non-convex optimization problem.
    We can convert it into a set of quadratic problems, by keeping either x_u or y_i fixed while optimizing the other.
    In that case, we can iteratively solve x and y by alternating between them until the algorithm converges.
    This is Alternating Least Squares.
    """

    RECOMMENDER_NAME = "IALSRecommender_implicit"

    def __init__(self, URM_train, verbose=True):
        super(IALSRecommender_implicit, self).__init__(URM_train, verbose=verbose)

    def fit(self, n_factors=50, regularization=0.001847510119137634, iterations=30, num_threads=2):
        self.n_factors = n_factors
        self.regularization = regularization
        self.iterations = iterations

        sparse_item_user = self.URM_train.T

        # Initialize the als model and fit it using the sparse item-user matrix
        model = implicit.als.AlternatingLeastSquares(factors=self.n_factors, regularization=self.regularization,
                                                     iterations=self.iterations, num_threads=num_threads)

        alpha_val = 2
        # Calculate the confidence by multiplying it by our alpha value.

        data_conf = (sparse_item_user * alpha_val).astype('double')

        # Fit the model
        model.fit(data_conf)

        # Get the user and item vectors from our trained model
        self.USER_factors = model.user_factors
        self.ITEM_factors = model.item_factors

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        """
        USER_factors is n_users x n_factors
        ITEM_factors is n_items x n_factors

        The prediction for cold users will always be -inf for ALL items

        :param user_id_array:
        :param items_to_compute:
        :return:
        """

        assert self.USER_factors.shape[1] == self.ITEM_factors.shape[1], \
            "{}: User and Item factors have inconsistent shape".format(self.RECOMMENDER_NAME)

        assert self.USER_factors.shape[0] > np.max(user_id_array), \
            "{}: Cold users not allowed. Users in trained model are {}, requested prediction for users up to {}".format(
                self.RECOMMENDER_NAME, self.USER_factors.shape[0], np.max(user_id_array))

        if items_to_compute is not None:
            item_scores = - np.ones((len(user_id_array), self.ITEM_factors.shape[0]), dtype=np.float32) * np.inf
            item_scores[:, items_to_compute] = np.dot(self.USER_factors[user_id_array],
                                                      np.transpose(self.ITEM_factors[items_to_compute, :]))

        else:
            item_factors_T = np.transpose(self.ITEM_factors)
            user_factors = self.USER_factors[user_id_array]
            item_scores = np.dot(user_factors, item_factors_T)

        # No need to select only the specific negative items or warm users because the -inf score will not change
        if self.use_bias:
            item_scores += self.ITEM_bias + self.GLOBAL_bias
            item_scores = np.transpose(np.transpose(item_scores) + self.USER_bias[user_id_array])

        return item_scores

In [8]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [9]:
recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_train)
recommender_SLIMElasticNet.fit(l1_ratio = 0.02, alpha = 0.0018503383172588782,  positive_only = True, topK = 600)

SLIMElasticNetRecommender: Processed 5306 (21.7%) in 5.00 min. Items per second: 17.68
SLIMElasticNetRecommender: Processed 11204 (45.7%) in 10.00 min. Items per second: 18.67
SLIMElasticNetRecommender: Processed 17225 (70.3%) in 15.00 min. Items per second: 19.14
SLIMElasticNetRecommender: Processed 23276 (95.0%) in 20.00 min. Items per second: 19.39
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 20.98 min. Items per second: 19.46


In [10]:
RP3beta_all = RP3betaRecommender(URM_train)
RP3beta_all.fit(alpha= 1.0, beta= 0.28666076265452467, topK= 57, implicit= True)

RP3betaRecommender: Similarity column 24507 (100.0%), 3024.32 column/sec. Elapsed time 8.10 sec


In [11]:
EASE_R = EASE_R_Recommender(URM_train)
#%%
EASE_R.fit(topK = 416, l2_norm = 115.67139771839786, normalize_matrix = False)

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 8.14 min


In [12]:
impl_IALS =  IALSRecommender_implicit(URM_train)
impl_IALS.fit(n_factors= 287, regularization= 40.37562274633393, iterations=70, num_threads=2)

  0%|          | 0/70 [00:00<?, ?it/s]

In [13]:
slim_matrix = recommender_SLIMElasticNet.W_sparse

In [14]:
rp3_matrix = RP3beta_all.W_sparse

In [15]:
ease_matrix = sps.csr_matrix(EASE_R.W_sparse)

In [16]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

for alpha in np.arange(0, 1.1, 0.5):
    for beta in np.arange(0, 1.1-alpha, 0.5):
        new_similarity = slim_matrix * alpha + ease_matrix * beta + rp3_matrix * (1-(alpha+beta))
        recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
        recommender_object.fit(new_similarity)
        result_df, _ = evaluator_valid.evaluateRecommender(recommender_object)
        print(f"Alpha: {alpha}, Beta: {beta}, Theta: {1-(alpha+beta)}, MAP: {result_df.loc[10]['MAP']}")

EvaluatorHoldout: Processed 41629 (100.0%) in 23.38 sec. Users per second: 1781
Alpha: 0.0, Beta: 0.0, Theta: 1.0, MAP: 0.033378569464889915
EvaluatorHoldout: Processed 41629 (100.0%) in 39.14 sec. Users per second: 1064
Alpha: 0.0, Beta: 0.5, Theta: 0.5, MAP: 0.035351897162652496
EvaluatorHoldout: Processed 41629 (100.0%) in 38.43 sec. Users per second: 1083
Alpha: 0.0, Beta: 1.0, Theta: 0.0, MAP: 0.034900350869565236
EvaluatorHoldout: Processed 41629 (100.0%) in 40.89 sec. Users per second: 1018
Alpha: 0.5, Beta: 0.0, Theta: 0.5, MAP: 0.03579859335696452
EvaluatorHoldout: Processed 41629 (100.0%) in 47.43 sec. Users per second: 878
Alpha: 0.5, Beta: 0.5, Theta: 0.0, MAP: 0.03546774474601058
EvaluatorHoldout: Processed 41629 (100.0%) in 40.06 sec. Users per second: 1039
Alpha: 1.0, Beta: 0.0, Theta: 0.0, MAP: 0.03565055286169016


In [17]:
#impl_IALS.W_sparse

In [18]:
"""from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
recommender_SLIM_BPR_Cython = SLIM_BPR_Cython(URM_all)
recommender_SLIM_BPR_Cython.fit(epochs= 615, sgd_mode= 'sgd', topK= 49, lambda_i= 0.0001, lambda_j= 0.0017579136035800475, learning_rate= 0.032671047315169746)"""

"from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython\nrecommender_SLIM_BPR_Cython = SLIM_BPR_Cython(URM_all)\nrecommender_SLIM_BPR_Cython.fit(epochs= 615, sgd_mode= 'sgd', topK= 49, lambda_i= 0.0001, lambda_j= 0.0017579136035800475, learning_rate= 0.032671047315169746)"

In [19]:
"""print("MAP of the starting models")

result_df, _ = evaluator_valid.evaluateRecommender(recommender_SLIMElasticNet)
print("SLIM - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_valid.evaluateRecommender(RP3beta_all)
print("RP3beta - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_valid.evaluateRecommender(EASE_R)
print("Ease - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_valid.evaluateRecommender(impl_IALS)
print("iIALS - MAP: {}".format(result_df.loc[10]["MAP"]))"""

'print("MAP of the starting models")\n\nresult_df, _ = evaluator_valid.evaluateRecommender(recommender_SLIMElasticNet)\nprint("SLIM - MAP: {}".format(result_df.loc[10]["MAP"]))\n\nresult_df, _ = evaluator_valid.evaluateRecommender(RP3beta_all)\nprint("RP3beta - MAP: {}".format(result_df.loc[10]["MAP"]))\n\nresult_df, _ = evaluator_valid.evaluateRecommender(EASE_R)\nprint("Ease - MAP: {}".format(result_df.loc[10]["MAP"]))\n\nresult_df, _ = evaluator_valid.evaluateRecommender(impl_IALS)\nprint("iIALS - MAP: {}".format(result_df.loc[10]["MAP"]))'

In [20]:
"""from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3, recommender_4):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        self.recommender_4 = recommender_4
        
        
        
    def fit(self, norm, alpha = 0.5, beta = 0.5, theta = 0):

        self.alpha = alpha
        self.beta = beta
        self.theta = theta
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)
        item_weights_4 = self.recommender_4._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        norm_item_weights_4 = LA.norm(item_weights_4, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_3 == 0:
            raise ValueError("Norm {} of item weights for recommender 3 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_4 == 0:
            raise ValueError("Norm {} of item weights for recommender 4 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * self.beta + item_weights_3 / norm_item_weights_3 * self.theta + item_weights_4 / norm_item_weights_4 * (1-(self.alpha+self.beta+self.theta))

        return item_weights"""

'from numpy import linalg as LA\nfrom Recommenders.BaseRecommender import BaseRecommender\n\nclass DifferentLossScoresHybridRecommender(BaseRecommender):\n\n    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"\n\n\n    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3, recommender_4):\n        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)\n\n        self.URM_train = sps.csr_matrix(URM_train)\n        self.recommender_1 = recommender_1\n        self.recommender_2 = recommender_2\n        self.recommender_3 = recommender_3\n        self.recommender_4 = recommender_4\n        \n        \n        \n    def fit(self, norm, alpha = 0.5, beta = 0.5, theta = 0):\n\n        self.alpha = alpha\n        self.beta = beta\n        self.theta = theta\n        self.norm = norm\n\n\n    def _compute_item_score(self, user_id_array, items_to_compute):\n        \n        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)\n      

In [21]:
"""import random

recommender_object = DifferentLossScoresHybridRecommender(URM_train, recommender_SLIMElasticNet, RP3beta_all, EASE_R, impl_IALS)

best_model = {
    "MAP" : 0,
    "alpha" : 0,
    "beta": 0,
    "theta": 0,
    "norm" : 0
}

n_searches = 0

for norm in [1,2]:
    while n_searches<50:
        alpha = random.uniform(0, 1)
        beta = random.uniform(0, 1-alpha)
        theta = random.uniform(0, 1- (alpha+beta))
        print("----")
        recommender_object.fit(norm, alpha, beta, theta)

        result_df, _ = evaluator_valid.evaluateRecommender(recommender_object)
        print("Norm: {}, Alpha: {}, Beta: {}, Theta: {}, Gamma: {}, Result: {}".format(norm, alpha, beta, theta, 1-(alpha+beta+theta), result_df.loc[10]["MAP"]))

        if result_df.loc[10]["MAP"] > best_model["MAP"]:
            best_model["MAP"] = result_df.loc[10]["MAP"]
            best_model["alpha"] = alpha
            best_model["beta"] = beta
            best_model["theta"] = theta
            best_model["norm"] = norm
                
        n_searches += 1
    n_searches = 30
print("----")
print("Best model has MAP: {} with alpha: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["norm"]))
"""

'import random\n\nrecommender_object = DifferentLossScoresHybridRecommender(URM_train, recommender_SLIMElasticNet, RP3beta_all, EASE_R, impl_IALS)\n\nbest_model = {\n    "MAP" : 0,\n    "alpha" : 0,\n    "beta": 0,\n    "theta": 0,\n    "norm" : 0\n}\n\nn_searches = 0\n\nfor norm in [1,2]:\n    while n_searches<50:\n        alpha = random.uniform(0, 1)\n        beta = random.uniform(0, 1-alpha)\n        theta = random.uniform(0, 1- (alpha+beta))\n        print("----")\n        recommender_object.fit(norm, alpha, beta, theta)\n\n        result_df, _ = evaluator_valid.evaluateRecommender(recommender_object)\n        print("Norm: {}, Alpha: {}, Beta: {}, Theta: {}, Gamma: {}, Result: {}".format(norm, alpha, beta, theta, 1-(alpha+beta+theta), result_df.loc[10]["MAP"]))\n\n        if result_df.loc[10]["MAP"] > best_model["MAP"]:\n            best_model["MAP"] = result_df.loc[10]["MAP"]\n            best_model["alpha"] = alpha\n            best_model["beta"] = beta\n            best_model["t

In [22]:
"""recommender = DifferentLossScoresHybridRecommender(URM_all, recommender_SLIMElasticNet, RP3beta_all, EASE_R, recommender_SLIM_BPR_Cython)
recommender.fit(1, 0.250566125884653, 0.1663158546388771, 0.573487903866556)"""

'recommender = DifferentLossScoresHybridRecommender(URM_all, recommender_SLIMElasticNet, RP3beta_all, EASE_R, recommender_SLIM_BPR_Cython)\nrecommender.fit(1, 0.250566125884653, 0.1663158546388771, 0.573487903866556)'

In [23]:
"""test_users = pd.read_csv('/kaggle/input/competition-data/data_target_users_test.csv')
test_users"""

"test_users = pd.read_csv('/kaggle/input/competition-data/data_target_users_test.csv')\ntest_users"

In [24]:
"""user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user,cutoff = 10, remove_seen_flag = True))
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submission4.csv', index=False)"""

'user_id = test_users[\'user_id\']\nrecommendations = []\nfor user in user_id:\n    recommendations.append(recommender.recommend(user,cutoff = 10, remove_seen_flag = True))\nfor index in range(len(recommendations)):\n    recommendations[index]=np.array(recommendations[index])\n    \ntest_users[\'item_list\']= recommendations\ntest_users[\'item_list\'] = pd.DataFrame([str(line).strip(\'[\').strip(\']\').replace("\'","") for line in test_users[\'item_list\']])\ntest_users.to_csv(\'submission4.csv\', index=False)'